# Data preprocessing

In [1]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `mail-classification` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-a

### Load Dataset from HuggingFace

In [2]:
import pandas as pd
from datasets import load_dataset
SEED = 42

# Load your dataset (replace with actual path or HF repo if needed)
dataset = load_dataset("Tobi-Bueck/customer-support-tickets")["train"]

# Convert to DataFrame
dataset_df = pd.DataFrame(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

aa_dataset-tickets-multi-lang-5-2-50-ver(…):   0%|          | 0.00/26.0M [00:00<?, ?B/s]

dataset-tickets-multi-lang-4-20k.csv:   0%|          | 0.00/18.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/48587 [00:00<?, ? examples/s]

In [3]:
dataset_df.head()

,subject,body,answer,type,queue,priority,language,version,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8
0,Wesentlicher Sicherheitsvorfall,"Sehr geehrtes Support-Team,\n\nich möchte eine...",Vielen Dank für die Meldung des kritischen Sic...,Incident,Technical Support,high,de,51.0,Security,Outage,Disruption,Data Breach,None,None,None,None
1,Account Disruption,"Dear Customer Support Team,\n\nI am writing to...","Thank you for reaching out, <name>. We are awa...",Incident,Technical Support,high,en,51.0,Account,Disruption,Outage,IT,Tech Support,None,None,None
2,Query About Smart Home System Integration Feat...,"Dear Customer Support Team,\n\nI hope this mes...",Thank you for your inquiry. Our products suppo...,Request,Returns and Exchanges,medium,en,51.0,Product,Feature,Tech Support,None,None,None,None,None
3,Inquiry Regarding Invoice Details,"Dear Customer Support Team,\n\nI hope this mes...",We appreciate you reaching out with your billi...,Request,Billing and Payments,low,en,51.0,Billing,Payment,Account,Documentation,Feedback,None,None,None
4,Question About Marketing Agency Software Compa...,"Dear Support Team,\n\nI hope this message reac...",Thank you for your inquiry. Our product suppor...,Problem,Sales and Pre-Sales,medium,en,51.0,Product,Feature,Feedback,Tech Support,None,None,None,None


### Feature Selection

In [4]:
selceted_cols = ["subject", "body", "queue", "type", "language"]
new_df = dataset_df[selceted_cols]


In [ ]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48587 entries, 0 to 48586
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   subject   43288 non-null  object
 1   body      48585 non-null  object
 2   queue     48587 non-null  object
 3   type      48587 non-null  object
 4   language  48587 non-null  object
dtypes: object(5)
memory usage: 1.9+ MB


## Distrubution of most important features

In [5]:
def label_language_dist(df):
    """Print distribution of languages grouped by queue and type."""
    # Precompute grouped distribution
    grouped = df.groupby(["queue", "type", "language"]).size().unstack(fill_value=0)

    # Iterate queue groups directly
    for queue_label, block in grouped.groupby(level=0):
        print(f"\n🔹 Queue: {queue_label}\n")
        print(block.droplevel(0))


In [6]:
label_language_dist(new_df)



🔹 Queue: Billing and Payments

language   de    en
type               
Change    105   169
Incident  478   688
Problem   432   618
Request   962  1422

🔹 Queue: Customer Service

language    de    en
type                
Change     198   347
Incident   826  1103
Problem    643   875
Request   1484  1944

🔹 Queue: General Inquiry

language  de   en
type             
Change    94  109
Incident  78  130
Problem   30   71
Request   62   94

🔹 Queue: Human Resources

language   de   en
type              
Change     20   39
Incident  204  280
Problem    35   65
Request   102  169

🔹 Queue: IT Support

language   de    en
type               
Change    390   586
Incident  909  1165
Problem   612   873
Request   481   709

🔹 Queue: Product Support

language    de    en
type                
Change     435   623
Incident  1529  2300
Problem    844  1190
Request    847  1192

🔹 Queue: Returns and Exchanges

language   de   en
type              
Change    137  174
Incident  430  553
Problem   210 

## Build a sebset dataframe

In [7]:
required_cols = ["queue", "type", "language"]

# Group by the combination and sample up to 100 rows from each group
subset_df= (
    new_df.groupby(required_cols, group_keys=False)
      .apply(lambda g: g.sample(n=min(len(g), 100), random_state=SEED))
      .reset_index(drop=True)
)


/tmp/ipython-input-3141817330.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(n=min(len(g), 100), random_state=SEED))


## Build an Augmentation Plan

In [8]:
import pandas as pd

def compute_augmentation_plan(df, threshold=100):
    counts = df.groupby(["queue", "type", "language"]).size()
    return (
        counts.reset_index(name="count")
              .assign(needed=lambda x: (threshold - x["count"]).clip(lower=0))
              .query("needed > 0")
              .sort_values("needed", ascending=False)
    )


In [9]:
df_aug_plan = pd.DataFrame(compute_augmentation_plan(subset_df))
df_aug_plan

,queue,type,language,count,needed
24,Human Resources,Change,de,20,80
20,General Inquiry,Problem,de,30,70
28,Human Resources,Problem,de,35,65
25,Human Resources,Change,en,39,61
68,Service Outages and Maintenance,Problem,de,55,45
22,General Inquiry,Request,de,62,38
29,Human Resources,Problem,en,65,35
69,Service Outages and Maintenance,Problem,en,69,31
21,General Inquiry,Problem,en,71,29
18,General Inquiry,Incident,de,78,22


# Dataset Augmentation

## 1: Device Detection

In [10]:
import torch

DEVICE = 0 if torch.cuda.is_available() else -1
print(f"Using {'GPU' if DEVICE == 0 else 'CPU'} for translation")


Using CPU for translation


## 2: Flexible Translation Loader

In [11]:
from transformers import pipeline

def get_translator(src_lang: str, tgt_lang: str, device: int = DEVICE):
    """ Loads a translation pipeline for the given source and target language."""
    model_name = f"Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}"
    return pipeline("translation", model=model_name, device=device)


## 3: Generalized Batch Translation Function

In [12]:
def batch_translate(texts, translator, batch_size=64):
    """
    Translates a list of texts using the provided translator pipeline in batches.
    Returns a list of translation dicts (each with 'translation_text').
    """
    results = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        results += translator(batch)
    return results


## 4: Generalized Sample Getter

In [13]:
def get_random_samples(df, queue, type_, language, needed):
    """
    Returns a list of randomly sampled bodies from the given language, queue, and type.
    If not enough samples exist, it duplicates the pool to meet the required count.
    """
    pool = df[
        (df["queue"] == queue) &
        (df["type"] == type_) &
        (df["language"] == language)
    ]

    texts = pool["body"].dropna().tolist()
    if not texts:
        return []

    while len(texts) < needed:
        texts += texts
    return random.sample(texts, needed)


## 5: Language-Agnostic Augmentation Loop

In [ ]:
# Used by MarianMT models, like the Helsinki-NLP to handle text preprocessing more accurately.
! pip install sacremoses



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 1.8 MB/s eta 0:00:00


## Augmentation Pipeline

In [ ]:
from tqdm import tqdm
import random


synthetic_examples = []

for _, row in tqdm(df_aug_plan.iterrows(), total=len(df_aug_plan), desc="Augmenting Samples"):
    queue = row["queue"]
    type_ = row["type"]
    language = row["language"]
    needed = row["needed"]

    # Define translation direction
    if language == "de":
        src_lang, tgt_lang = "en", "de"
    elif language == "en":
        src_lang, tgt_lang = "de", "en"
    else:
        continue  # Skip unsupported languages

    # Load translator
    translator = get_translator(src_lang, tgt_lang, device=DEVICE)

    # Sample source texts
    source_bodies = get_random_samples(df, queue, type_, src_lang, needed)

    source_texts = df[
        (df["body"].isin(source_bodies)) &
        (df["language"] == src_lang)
    ][["subject", "body"]].drop_duplicates().to_dict(orient="records")

    while len(source_texts) < needed:
        source_texts += source_texts
    source_texts = random.sample(source_texts, needed)

    subjects_src = [t["subject"] for t in source_texts if isinstance(t["subject"], str) and t["subject"].strip()]
    bodies_src = [t["body"] for t in source_texts if isinstance(t["body"], str) and t["body"].strip()]

    subjects_tgt = batch_translate(subjects_src, translator)
    bodies_tgt = batch_translate(bodies_src, translator)

    min_len = min(len(subjects_tgt), len(bodies_tgt), needed)
    for i in range(min_len):
        synthetic_examples.append({
            "subject": subjects_tgt[i]["translation_text"],
            "body": bodies_tgt[i]["translation_text"],
            "queue": queue,
            "type": type_,
            "language": tgt_lang,
            "source": "synthetic",
            "source_subject": subjects_src[i],
            "source_body": bodies_src[i],
            "seed": SEED
        })


Augmenting Samples:   0%|          | 0/13 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Augmenting Samples: 100%|██████████| 13/13 [39:11<00:00, 180.89s/it]


In [ ]:
synthetic_df = pd.DataFrame(synthetic_examples)

synthetic_df.to_csv("synthetic_examples.csv", index=False)


In [ ]:
synthetic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subject         419 non-null    object
 1   body            419 non-null    object
 2   queue           419 non-null    object
 3   type            419 non-null    object
 4   language        419 non-null    object
 5   source          419 non-null    object
 6   source_subject  419 non-null    object
 7   source_body     419 non-null    object
 8   seed            419 non-null    int64 
dtypes: int64(1), object(8)
memory usage: 29.6+ KB


In [ ]:
subset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7505 entries, 0 to 7504
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   subject   6645 non-null   object
 1   body      7505 non-null   object
 2   queue     7505 non-null   object
 3   type      7505 non-null   object
 4   language  7505 non-null   object
dtypes: object(5)
memory usage: 293.3+ KB


In [ ]:
# Merge subset dataframe and augmented dataframe

synthetic_selected = synthetic_df[["subject", "body", "queue", "type", "language"]].copy()

augmented_df = pd.concat([subset_df, synthetic_selected], ignore_index=True)

augmented_df = augmented_df.sample(frac=1, random_state=42).reset_index(drop=True)



In [ ]:
augmented_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7924 entries, 0 to 7923
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   subject   7064 non-null   object
 1   body      7924 non-null   object
 2   queue     7924 non-null   object
 3   type      7924 non-null   object
 4   language  7924 non-null   object
dtypes: object(5)
memory usage: 309.7+ KB


In [ ]:
augmented_df['subject'] = augmented_df['subject'].fillna('').astype(str)


In [ ]:
augmented_df[:10]

,subject,body,queue,type,language
0,Investment Tracking Dashboard Update Problem,"The dashboard update has failed, which might b...",Sales and Pre-Sales,Incident,en
1,Healthcare Provider Data Breach,"A data breach has been detected, which could e...",Billing and Payments,Incident,en
2,Protecting Medical Data in Healthcare Systems,Can you offer advice on effectively safeguardi...,Product Support,Request,en
3,Problem mit ungenauen Investitionsprognosen,"Sehr geehrte Kundensupport, ich schreibe, um I...",Customer Service,Problem,de
4,Concerns Over Team Communication,"Customer Support, <br><br>I am reporting an is...",Human Resources,Incident,en
5,None,"Customer Support, I am reaching out to inquire...",Technical Support,Request,en
6,Analysieren Abrechnungsunterschiede mit digita...,"Sehr geehrter Kunde, wir möchten uns bei der B...",Billing and Payments,Incident,de
7,Support Required for Potential Data Breach,A potential data breach has been identified in...,Human Resources,Incident,en
8,Verschlusssicherung medizinischer Daten in Kra...,Könnten Sie Informationen zu Lösungen für die ...,Sales and Pre-Sales,Request,de
9,Sicherheitslücke in medizinischen IT-Systemen ...,Medizinische Daten wurden durch veraltete Soft...,Returns and Exchanges,Problem,de


In [ ]:
label_language_dist(augmented_df)


🔹 Queue: Billing and Payments

language   de   en
type              
Change    100  100
Incident  100  100
Problem   100  100
Request   100  100

🔹 Queue: Customer Service

language   de   en
type              
Change    100  100
Incident  100  100
Problem   100  100
Request   100  100

🔹 Queue: General Inquiry

language   de   en
type              
Change    100  100
Incident   96  100
Problem    77   96
Request    93   99

🔹 Queue: Human Resources

language   de   en
type              
Change     93   92
Incident  100  100
Problem    86   95
Request   100  100

🔹 Queue: IT Support

language   de   en
type              
Change    100  100
Incident  100  100
Problem   100  100
Request   100  100

🔹 Queue: Product Support

language   de   en
type              
Change    100  100
Incident  100  100
Problem   100  100
Request   100  100

🔹 Queue: Returns and Exchanges

language   de   en
type              
Change    100  100
Incident  100  100
Problem   100  100
Request   100  100

🔹 Que

# Push Dataset to HuggingFace Hub

In [ ]:
from datasets import Dataset
augmented_dataset = Dataset.from_pandas(augmented_df)

In [ ]:
augmented_dataset.push_to_hub("ale-dp/bilingual-ticket-classification", private=True)


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :  83%|########2 | 1.58MB / 1.90MB            

CommitInfo(commit_url='https://huggingface.co/datasets/ale-dp/bilingual-ticket-classification/commit/2c72662a3b459c641bf361d22faa174b88d38729', commit_message='Upload dataset', commit_description='', oid='2c72662a3b459c641bf361d22faa174b88d38729', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ale-dp/bilingual-ticket-classification', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ale-dp/bilingual-ticket-classification'), pr_revision=None, pr_num=None)

### Dataset-Readme Script helper

In [ ]:
def extract_readme_info(dataset_obj):
    # Check if it's a DatasetDict or Dataset
    if hasattr(dataset_obj, "keys"):
        first_split_name = list(dataset_obj.keys())[0]
        dataset = dataset_obj[first_split_name]
    else:
        dataset = dataset_obj

    info = {}

    # General info
    info['num_rows'] = len(dataset)
    info['num_features'] = len(dataset.column_names)

    # Features and types
    features_info = {}
    for feature_name, feature_obj in dataset.features.items():
        features_info[feature_name] = str(feature_obj)
    info['features'] = features_info

    # Sample data (first 5 rows)
    samples = dataset.select(range(min(5, len(dataset))))
    info['sample_data'] = samples

    return info


if __name__ == "__main__":
    readme_info = extract_readme_info(augmented_dataset)

    # Print summary info
    print(f"Split: {list(augmented_dataset.keys())[0] if hasattr(augmented_dataset, 'keys') else 'N/A'}")
    print(f"Number of rows: {readme_info['num_rows']}")
    print(f"Number of features: {readme_info['num_features']}\n")

    print("Features and types:")
    for fname, ftype in readme_info['features'].items():
        print(f"- {fname}: {ftype}")
    print("\nSample data (first 5 rows):")
    print(readme_info['sample_data'])

Split: N/A
Number of rows: 7924
Number of features: 5

Features and types:
- subject: Value('string')
- body: Value('string')
- queue: Value('string')
- type: Value('string')
- language: Value('string')

Sample data (first 5 rows):
Dataset({
    features: ['subject', 'body', 'queue', 'type', 'language'],
    num_rows: 5
})


In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_file(
    path_or_fileobj="README.md", # Path to the file to upload
    path_in_repo="README.md",
    repo_id="ale-dp/bilingual-ticket-classification",  # Replace with your dataset repo ID
    repo_type="dataset",
    commit_message="Add README.md"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/hf_api.py:9706: UserWarning: Warnings while validating metadata in README.md:
- empty or missing yaml metadata in repo card
  warnings.warn(f"Warnings while validating metadata in README.md:\n{message}")


CommitInfo(commit_url='https://huggingface.co/datasets/ale-dp/bilingual-ticket-classification/commit/0ccc857495fc0fac8ebf1791aef9ff661d841b95', commit_message='Add README.md', commit_description='', oid='0ccc857495fc0fac8ebf1791aef9ff661d841b95', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ale-dp/bilingual-ticket-classification', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ale-dp/bilingual-ticket-classification'), pr_revision=None, pr_num=None)